# Dekamarkt Product Scraper v2.0
## Sitemap-Based Approach with Detailed Product GraphQL

This notebook implements a comprehensive dekamarkt scraper that:
- **Discovers products** from sitemap: `https://www.dekamarkt.nl/products-sitemap.xml`
- **Tracks lastmod dates** to scrape only updated products (like dirk.py)
- **Uses detailed product GraphQL** to get complete product information
- **Saves comprehensive data** including nutritional info, descriptions, images, etc.

### Features:
- ✅ Sitemap-based product discovery
- ✅ LastMod date tracking for incremental updates
- ✅ Detailed product information via GraphQL
- ✅ Complete nutritional and ingredient data
- ✅ Image URLs and product descriptions
- ✅ CSV export with comprehensive data

In [1]:
# 📦 IMPORTS AND CONFIGURATION
import time
import requests
import pandas as pd
from datetime import datetime, timedelta
import xml.etree.ElementTree as ET
from urllib.parse import urlparse, parse_qs
import re
import json
from typing import List, Dict, Any, Optional
import os
import pickle

# API Configuration
ENDPOINT = 'https://web-deka-gateway.detailresult.nl/graphql'
SITEMAP_URL = 'https://www.dekamarkt.nl/products-sitemap.xml'
STORE_ID = 283  # Default store ID

# Headers for GraphQL requests
HEADERS = {
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.5',
    'api_key': '6d3a42a3-6d93-4f98-838d-bcc0ab2307fd',
    'content-type': 'application/json',
    'origin': 'https://www.dekamarkt.nl',
    'referer': 'https://www.dekamarkt.nl/',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/139.0.0.0 Safari/537.36'
}

# Detailed product GraphQL query template
PRODUCT_QUERY_TEMPLATE = '''{
  "query": "query { product(productId: %s) { productId headerText brand packaging isWeightProduct maxAmount department webgroup additionalDescription description declarations { contactInformation { contactName contactAdress } nutritionalInformation { standardPackagingUnit soldOrPrepared nutritionalValues { text value nutritionalSubValues { text value } } } storageInstructions cookingInstructions instructionsForUse ingredients allergiesInformation { text } } logos { position description image } images { image rankNumber mainImage } productAssortment(storeId: %s) { productId storeId normalPrice offerPrice isSingleUsePlastic productNumber startDate endDate productOffer { productId textPriceSign endDate startDate disclaimerStartDate disclaimerEndDate } productInformation { productId headerText subText packaging image isWeightProduct department webgroup brand logos { position description image } } } } }",
  "variables": {}
}'''

# File for storing last update tracking
LAST_UPDATE_FILE = 'dekamarkt_last_update.pkl'

print("✅ Configuration loaded successfully!")

✅ Configuration loaded successfully!


In [32]:
# 🗺️ SITEMAP DISCOVERY FUNCTIONS

def fetch_sitemap(url: str) -> str:
    """Fetch sitemap XML content from URL."""
    try:
        response = requests.get(url, timeout=30)
        response.raise_for_status()
        return response.text
    except Exception as e:
        print(f"❌ Error fetching sitemap: {e}")
        return None

def parse_product_urls_from_sitemap(sitemap_content: str) -> List[Dict[str, Any]]:
    """
    Parse sitemap XML to extract product IDs and lastmod dates.
    Returns list of dicts with product_id, url, and lastmod.
    """
    if not sitemap_content:
        return []
    
    products = []
    try:
        root = ET.fromstring(sitemap_content)
        
        # Handle namespace
        ns = {'sitemap': 'http://www.sitemaps.org/schemas/sitemap/0.9'}
        
        for url_elem in root.findall('.//sitemap:url', ns):
            loc_elem = url_elem.find('sitemap:loc', ns)
            lastmod_elem = url_elem.find('sitemap:lastmod', ns)
            
            if loc_elem is not None:
                url = loc_elem.text
                # Extract product ID from URL (assuming format like /product/12345-product-name)
                product_id_match = re.search(r'/product/(\d+)', url)
                
                if product_id_match:
                    product_id = product_id_match.group(1)
                    lastmod = lastmod_elem.text if lastmod_elem is not None else None
                    
                    # Parse lastmod date if available
                    lastmod_date = None
                    if lastmod:
                        try:
                            # Handle different date formats
                            if 'T' in lastmod:
                                lastmod_date = datetime.fromisoformat(lastmod.replace('Z', '+00:00'))
                            else:
                                lastmod_date = datetime.strptime(lastmod, '%Y-%m-%d')
                        except:
                            pass
                    
                    products.append({
                        'product_id': product_id,
                        'url': url,
                        'lastmod': lastmod,
                        'lastmod_date': lastmod_date
                    })
    except Exception as e:
        print(f"❌ Error parsing sitemap: {e}")
    
    return products

def get_last_update_date() -> Optional[datetime]:
    """Get the last update date from storage."""
    if os.path.exists(LAST_UPDATE_FILE):
        try:
            with open(LAST_UPDATE_FILE, 'rb') as f:
                return pickle.load(f)
        except:
            pass
    return None

def save_last_update_date(date: datetime):
    """Save the last update date to storage."""
    try:
        with open(LAST_UPDATE_FILE, 'wb') as f:
            pickle.dump(date, f)
        print(f"📅 Saved last update date: {date}")
    except Exception as e:
        print(f"❌ Error saving last update date: {e}")

def filter_products_by_date(products: List[Dict], last_update: Optional[datetime] = None) -> List[Dict]:
    """
    Filter products to only include those with lastmod after the last update date.
    If no last update date, return all products.
    """
    if not last_update:
        print("🔄 No previous update date found - will scrape all products")
        return products
    
    filtered = []
    for product in products:
        if product['lastmod_date'] and product['lastmod_date'] > last_update:
            filtered.append(product)
    
    print(f"📊 Found {len(filtered)} products updated after {last_update.strftime('%Y-%m-%d %H:%M:%S')}")
    return filtered

print("✅ Sitemap parsing functions ready!")

✅ Sitemap parsing functions ready!


In [33]:
# 🛒 DETAILED PRODUCT QUERY FUNCTIONS

def fetch_product_details(product_id: str, store_id: int = STORE_ID) -> Optional[Dict]:
    """
    Fetch detailed product information using the comprehensive GraphQL query.
    """
    query = PRODUCT_QUERY_TEMPLATE % (product_id, store_id)
    
    try:
        response = requests.post(
            ENDPOINT,
            headers=HEADERS,
            data=query,
            timeout=30
        )
        response.raise_for_status()
        
        data = response.json()
        if 'errors' in data:
            print(f"❌ GraphQL errors for product {product_id}: {data['errors']}")
            return None
            
        product_data = data.get('data', {}).get('product')
        if not product_data:
            print(f"⚠️ No product data found for ID {product_id}")
            return None
            
        return product_data
        
    except Exception as e:
        print(f"❌ Error fetching product {product_id}: {e}")
        return None

def extract_comprehensive_product_info(product_data: Dict) -> Dict[str, Any]:
    """
    Extract comprehensive product information from the detailed GraphQL response.
    """
    if not product_data:
        return {}
    
    # Basic product info
    basic_info = {
        'product_id': product_data.get('productId'),
        'header_text': product_data.get('headerText'),
        'brand': product_data.get('brand'),
        'packaging': product_data.get('packaging'),
        'is_weight_product': product_data.get('isWeightProduct'),
        'max_amount': product_data.get('maxAmount'),
        'department': product_data.get('department'),
        'webgroup': product_data.get('webgroup'),
        'description': product_data.get('description'),
        'additional_description': product_data.get('additionalDescription')
    }
    
    # Product assortment (pricing and offer info)
    assortment = product_data.get('productAssortment', {})
    if assortment:
        basic_info.update({
            'normal_price': assortment.get('normalPrice'),
            'offer_price': assortment.get('offerPrice'),
            'is_single_use_plastic': assortment.get('isSingleUsePlastic'),
            'product_number': assortment.get('productNumber'),
            'start_date': assortment.get('startDate'),
            'end_date': assortment.get('endDate')
        })
        
        # Product offer details
        offer = assortment.get('productOffer', {})
        if offer:
            basic_info.update({
                'offer_text_price_sign': offer.get('textPriceSign'),
                'offer_start_date': offer.get('startDate'),
                'offer_end_date': offer.get('endDate'),
                'disclaimer_start_date': offer.get('disclaimerStartDate'),
                'disclaimer_end_date': offer.get('disclaimerEndDate')
            })
    
    # Images
    images = product_data.get('images', [])
    if images:
        main_images = [img['image'] for img in images if img.get('mainImage')]
        all_images = [img['image'] for img in images]
        basic_info.update({
            'main_image': main_images[0] if main_images else None,
            'all_images': ', '.join(all_images) if all_images else None,
            'image_count': len(all_images)
        })
    
    # Logos
    logos = product_data.get('logos', [])
    if logos:
        logo_descriptions = [logo.get('description', '') for logo in logos if logo.get('description')]
        basic_info['logos'] = ', '.join(logo_descriptions) if logo_descriptions else None
    
    # Declarations (nutritional and other info)
    declarations = product_data.get('declarations', {})
    if declarations:
        # Contact information
        contact = declarations.get('contactInformation', {})
        if contact:
            basic_info.update({
                'contact_name': contact.get('contactName'),
                'contact_address': contact.get('contactAdress')
            })
        
        # Nutritional information
        nutrition = declarations.get('nutritionalInformation', {})
        if nutrition:
            basic_info.update({
                'standard_packaging_unit': nutrition.get('standardPackagingUnit'),
                'sold_or_prepared': nutrition.get('soldOrPrepared')
            })
            
            # Nutritional values
            nutritional_values = nutrition.get('nutritionalValues', [])
            if nutritional_values:
                nutrition_text = []
                for nv in nutritional_values:
                    text = nv.get('text', '')
                    value = nv.get('value', '')
                    if text and value:
                        nutrition_text.append(f"{text}: {value}")
                        
                        # Sub-values
                        sub_values = nv.get('nutritionalSubValues', [])
                        for sv in sub_values:
                            sub_text = sv.get('text', '')
                            sub_value = sv.get('value', '')
                            if sub_text and sub_value:
                                nutrition_text.append(f"  - {sub_text}: {sub_value}")
                
                basic_info['nutritional_values'] = '; '.join(nutrition_text) if nutrition_text else None
        
        # Other declarations
        basic_info.update({
            'storage_instructions': declarations.get('storageInstructions'),
            'cooking_instructions': declarations.get('cookingInstructions'),
            'instructions_for_use': declarations.get('instructionsForUse'),
            'ingredients': declarations.get('ingredients')
        })
        
        # Allergies information
        allergies = declarations.get('allergiesInformation', {})
        if allergies and isinstance(allergies, dict):
            basic_info['allergies_info'] = allergies.get('text')
    
    return basic_info

print("✅ Detailed product query functions ready!")

✅ Detailed product query functions ready!


In [34]:
# 🚀 MAIN SCRAPING ORCHESTRATION

def scrape_dekamarkt_comprehensive(
    store_id: int = STORE_ID,
    delay_between_requests: float = 0.1,
    max_products: int = None,
    force_full_scrape: bool = False
) -> pd.DataFrame:
    """
    Comprehensive Dekamarkt scraper using sitemap discovery and detailed product queries.
    
    Args:
        store_id: Store ID for pricing/availability
        delay_between_requests: Delay between API calls (seconds)
        max_products: Maximum number of products to scrape (for testing)
        force_full_scrape: If True, ignore lastmod dates and scrape all products
    
    Returns:
        DataFrame with comprehensive product information
    """
    
    print("🏁 Starting Dekamarkt Comprehensive Scraper v2.0")
    print("=" * 60)
    
    # Step 1: Fetch and parse sitemap
    print("📥 Fetching product sitemap...")
    sitemap_content = fetch_sitemap(SITEMAP_URL)
    if not sitemap_content:
        print("❌ Failed to fetch sitemap")
        return pd.DataFrame()
    
    # Step 2: Extract product URLs and lastmod dates
    print("🔍 Parsing product URLs from sitemap...")
    products = parse_product_urls_from_sitemap(sitemap_content)
    if not products:
        print("❌ No products found in sitemap")
        return pd.DataFrame()
    
    print(f"📊 Found {len(products)} products in sitemap")
    
    # Step 3: Filter by lastmod date (unless force_full_scrape)
    if not force_full_scrape:
        last_update = get_last_update_date()
        products = filter_products_by_date(products, last_update)
    else:
        print("🔄 Force full scrape enabled - processing all products")
    
    if not products:
        print("✅ No new products to scrape")
        return pd.DataFrame()
    
    # Step 4: Limit products for testing if specified
    if max_products and len(products) > max_products:
        products = products[:max_products]
        print(f"🎯 Limited to first {max_products} products for testing")
    
    # Step 5: Scrape detailed product information
    print(f"🛒 Scraping detailed information for {len(products)} products...")
    scraped_products = []
    errors = 0
    
    for i, product_info in enumerate(products, 1):
        product_id = product_info['product_id']
        
        if i % 50 == 0:
            print(f"🔄 Progress: {i}/{len(products)} products processed...")
        
        # Fetch detailed product data
        product_data = fetch_product_details(product_id, store_id)
        if product_data:
            # Extract comprehensive information
            extracted_info = extract_comprehensive_product_info(product_data)
            if extracted_info:
                # Add sitemap metadata
                extracted_info.update({
                    'sitemap_url': product_info['url'],
                    'lastmod': product_info['lastmod'],
                    'scraped_at': datetime.now().isoformat()
                })
                scraped_products.append(extracted_info)
            else:
                errors += 1
        else:
            errors += 1
        
        # Rate limiting
        if delay_between_requests > 0:
            time.sleep(delay_between_requests)
    
    print(f"✅ Scraping completed!")
    print(f"📊 Successfully scraped: {len(scraped_products)} products")
    print(f"❌ Errors encountered: {errors} products")
    
    # Step 6: Create DataFrame and save results
    if scraped_products:
        df = pd.DataFrame(scraped_products)
        
        # Save last update date
        current_time = datetime.now()
        save_last_update_date(current_time)
        
        print(f"💾 Results saved with {len(df)} products")
        print(f"📅 Next scrape will check for products updated after: {current_time}")
        
        return df
    else:
        print("❌ No products successfully scraped")
        return pd.DataFrame()

def save_to_csv(df: pd.DataFrame, filename: str = None) -> str:
    """Save DataFrame to CSV with timestamp."""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"dekamarkt_products_comprehensive_{timestamp}.csv"
    
    df.to_csv(filename, index=False)
    print(f"💾 Saved {len(df)} products to: {filename}")
    return filename

print("✅ Main scraping orchestration ready!")

✅ Main scraping orchestration ready!


In [35]:
# 🧪 TEST THE NEW COMPREHENSIVE SCRAPER

print("🧪 Testing Dekamarkt Comprehensive Scraper v2.0")
print("=" * 60)

# Test with a small sample first
print("📋 Starting test run with limited products...")

# Run the comprehensive scraper
df_comprehensive = scrape_dekamarkt_comprehensive(
    store_id=283,
    delay_between_requests=0.1,  # 100ms delay
    max_products=5,  # Test with just 5 products first
    force_full_scrape=True  # Ignore date filtering for test
)

if not df_comprehensive.empty:
    print(f"\n📊 SAMPLE RESULTS OVERVIEW:")
    print(f"Total products scraped: {len(df_comprehensive)}")
    print(f"Columns available: {len(df_comprehensive.columns)}")
    print(f"Sample columns: {list(df_comprehensive.columns[:10])}")
    
    # Show a sample product
    if len(df_comprehensive) > 0:
        print(f"\n🔍 SAMPLE PRODUCT DETAILS:")
        sample = df_comprehensive.iloc[0]
        key_fields = ['product_id', 'header_text', 'brand', 'normal_price', 'offer_price', 
                     'department', 'description', 'nutritional_values', 'ingredients']
        
        for field in key_fields:
            if field in sample and pd.notna(sample[field]):
                value = str(sample[field])
                if len(value) > 100:
                    value = value[:100] + "..."
                print(f"  {field}: {value}")
    
    # Show column summary
    print(f"\n📋 ALL AVAILABLE COLUMNS:")
    for i, col in enumerate(df_comprehensive.columns, 1):
        print(f"  {i:2d}. {col}")
        
else:
    print("❌ Test failed - no data retrieved")

print(f"\n{'='*60}")
print("🧪 Test completed! Review results above.")

🧪 Testing Dekamarkt Comprehensive Scraper v2.0
📋 Starting test run with limited products...
🏁 Starting Dekamarkt Comprehensive Scraper v2.0
📥 Fetching product sitemap...
🔍 Parsing product URLs from sitemap...
❌ No products found in sitemap
❌ Test failed - no data retrieved

🧪 Test completed! Review results above.


In [36]:
# 🔍 DEBUG SITEMAP STRUCTURE

print("🔍 Investigating sitemap structure...")

# Fetch sitemap content
sitemap_content = fetch_sitemap(SITEMAP_URL)

if sitemap_content:
    print(f"✅ Sitemap fetched successfully ({len(sitemap_content)} characters)")
    
    # Show first 2000 characters to understand structure
    print("\n📋 First 2000 characters of sitemap:")
    print("-" * 60)
    print(sitemap_content[:2000])
    print("-" * 60)
    
    # Try to parse XML structure
    try:
        root = ET.fromstring(sitemap_content)
        print(f"\n🔍 XML Root tag: {root.tag}")
        print(f"🔍 XML Root attributes: {root.attrib}")
        
        # Check namespaces
        namespaces = {}
        for event, elem in ET.iterparse(ET.StringIO(sitemap_content), events=("start-ns",)):
            namespaces[event] = elem
        print(f"🔍 Namespaces found: {namespaces}")
        
        # List all direct children
        print(f"\n🔍 Direct children of root:")
        for i, child in enumerate(root):
            print(f"  {i+1}. Tag: {child.tag}, Attributes: {child.attrib}")
            if i < 5:  # Show details for first 5 elements
                for subchild in child:
                    print(f"     - {subchild.tag}: {subchild.text[:100] if subchild.text else 'None'}")
            if i >= 10:  # Limit output
                print(f"  ... and {len(root) - 10} more elements")
                break
                
    except Exception as e:
        print(f"❌ Error parsing XML: {e}")
        
    # Look for product URLs manually
    print(f"\n🔍 Searching for product URL patterns...")
    product_matches = re.findall(r'<loc>([^<]*product[^<]*)</loc>', sitemap_content)
    print(f"Found {len(product_matches)} URLs containing 'product'")
    if product_matches:
        print("First 5 product URLs:")
        for i, url in enumerate(product_matches[:5]):
            print(f"  {i+1}. {url}")
    
else:
    print("❌ Failed to fetch sitemap")

🔍 Investigating sitemap structure...
✅ Sitemap fetched successfully (2148131 characters)

📋 First 2000 characters of sitemap:
------------------------------------------------------------
<?xml version="1.0" encoding="utf-8"?><urlset xmlns="http://www.sitemaps.org/schemas/sitemap/0.9"><url><loc>https://www.dekamarkt.nl/boodschappen/producten/dranken-sap-koffie-thee/bier/heineken-pilsener-krat-24-x-300-ml/6</loc><lastmod>2024-10-03T00:55+02:00</lastmod></url><url><loc>https://www.dekamarkt.nl/boodschappen/producten/kind-drogisterij/bad-douche-zeep/trida-zeepblok-soft-en-sensitive-3-stuks/8</loc><lastmod>2024-10-03T00:55+02:00</lastmod></url><url><loc>https://www.dekamarkt.nl/boodschappen/producten/aardappelen-groente-fruit/groente/bio-biologische-rode-bieten-gekookt-500-g/9</loc><lastmod>2024-10-03T00:55+02:00</lastmod></url><url><loc>https://www.dekamarkt.nl/boodschappen/producten/voorraadkast/oliën-sauzen-mixen/maggi-doseerjus-naturel-180-g/10</loc><lastmod>2024-10-03T00:55+02:00</last

In [37]:
# 🔧 FIXED SITEMAP PARSING FUNCTION

def parse_product_urls_from_sitemap_fixed(sitemap_content: str) -> List[Dict[str, Any]]:
    """
    Parse sitemap XML to extract product IDs and lastmod dates.
    Fixed version for dekamarkt URL format.
    """
    if not sitemap_content:
        return []
    
    products = []
    try:
        root = ET.fromstring(sitemap_content)
        
        # Handle namespace
        ns = {'sitemap': 'http://www.sitemaps.org/schemas/sitemap/0.9'}
        
        for url_elem in root.findall('.//sitemap:url', ns):
            loc_elem = url_elem.find('sitemap:loc', ns)
            lastmod_elem = url_elem.find('sitemap:lastmod', ns)
            
            if loc_elem is not None:
                url = loc_elem.text
                # Extract product ID from URL (dekamarkt format: ends with /product-id)
                # Example: https://www.dekamarkt.nl/boodschappen/producten/.../6
                if '/producten/' in url:
                    product_id_match = re.search(r'/(\d+)$', url)
                    
                    if product_id_match:
                        product_id = product_id_match.group(1)
                        lastmod = lastmod_elem.text if lastmod_elem is not None else None
                        
                        # Parse lastmod date if available
                        lastmod_date = None
                        if lastmod:
                            try:
                                # Handle different date formats
                                if 'T' in lastmod:
                                    # Remove timezone info for simpler parsing
                                    lastmod_clean = re.sub(r'[+-]\d{2}:\d{2}$', '', lastmod)
                                    lastmod_date = datetime.fromisoformat(lastmod_clean)
                                else:
                                    lastmod_date = datetime.strptime(lastmod, '%Y-%m-%d')
                            except Exception as e:
                                print(f"⚠️ Date parsing error for {lastmod}: {e}")
                                pass
                        
                        products.append({
                            'product_id': product_id,
                            'url': url,
                            'lastmod': lastmod,
                            'lastmod_date': lastmod_date
                        })
    except Exception as e:
        print(f"❌ Error parsing sitemap: {e}")
    
    return products

# Test the fixed function
print("🔧 Testing fixed sitemap parsing...")
test_products = parse_product_urls_from_sitemap_fixed(sitemap_content)

if test_products:
    print(f"✅ Successfully parsed {len(test_products)} products")
    print(f"\n📋 Sample products:")
    for i, product in enumerate(test_products[:5]):
        print(f"  {i+1}. ID: {product['product_id']}, LastMod: {product['lastmod']}")
        print(f"     URL: {product['url']}")
else:
    print("❌ Still no products found")

🔧 Testing fixed sitemap parsing...
✅ Successfully parsed 11611 products

📋 Sample products:
  1. ID: 6, LastMod: 2024-10-03T00:55+02:00
     URL: https://www.dekamarkt.nl/boodschappen/producten/dranken-sap-koffie-thee/bier/heineken-pilsener-krat-24-x-300-ml/6
  2. ID: 8, LastMod: 2024-10-03T00:55+02:00
     URL: https://www.dekamarkt.nl/boodschappen/producten/kind-drogisterij/bad-douche-zeep/trida-zeepblok-soft-en-sensitive-3-stuks/8
  3. ID: 9, LastMod: 2024-10-03T00:55+02:00
     URL: https://www.dekamarkt.nl/boodschappen/producten/aardappelen-groente-fruit/groente/bio-biologische-rode-bieten-gekookt-500-g/9
  4. ID: 10, LastMod: 2024-10-03T00:55+02:00
     URL: https://www.dekamarkt.nl/boodschappen/producten/voorraadkast/oliën-sauzen-mixen/maggi-doseerjus-naturel-180-g/10
  5. ID: 11, LastMod: 2024-10-03T00:55+02:00
     URL: https://www.dekamarkt.nl/boodschappen/producten/dranken-sap-koffie-thee/aperitieven-mixen/tokkelroom-naturel-500-ml/11


In [38]:
# 🚀 UPDATED MAIN SCRAPING FUNCTION WITH FIXED PARSER

def scrape_dekamarkt_comprehensive_v2(
    store_id: int = STORE_ID,
    delay_between_requests: float = 0.1,
    max_products: int = None,
    force_full_scrape: bool = False
) -> pd.DataFrame:
    """
    Comprehensive Dekamarkt scraper using sitemap discovery and detailed product queries.
    Updated with fixed sitemap parser.
    """
    
    print("🏁 Starting Dekamarkt Comprehensive Scraper v2.0 (Fixed)")
    print("=" * 60)
    
    # Step 1: Fetch and parse sitemap
    print("📥 Fetching product sitemap...")
    sitemap_content = fetch_sitemap(SITEMAP_URL)
    if not sitemap_content:
        print("❌ Failed to fetch sitemap")
        return pd.DataFrame()
    
    # Step 2: Extract product URLs and lastmod dates (using fixed parser)
    print("🔍 Parsing product URLs from sitemap...")
    products = parse_product_urls_from_sitemap_fixed(sitemap_content)
    if not products:
        print("❌ No products found in sitemap")
        return pd.DataFrame()
    
    print(f"📊 Found {len(products)} products in sitemap")
    
    # Step 3: Filter by lastmod date (unless force_full_scrape)
    if not force_full_scrape:
        last_update = get_last_update_date()
        products = filter_products_by_date(products, last_update)
    else:
        print("🔄 Force full scrape enabled - processing all products")
    
    if not products:
        print("✅ No new products to scrape")
        return pd.DataFrame()
    
    # Step 4: Limit products for testing if specified
    if max_products and len(products) > max_products:
        products = products[:max_products]
        print(f"🎯 Limited to first {max_products} products for testing")
    
    # Step 5: Scrape detailed product information
    print(f"🛒 Scraping detailed information for {len(products)} products...")
    scraped_products = []
    errors = 0
    
    for i, product_info in enumerate(products, 1):
        product_id = product_info['product_id']
        
        if i % 50 == 0:
            print(f"🔄 Progress: {i}/{len(products)} products processed...")
        
        # Fetch detailed product data
        product_data = fetch_product_details(product_id, store_id)
        if product_data:
            # Extract comprehensive information
            extracted_info = extract_comprehensive_product_info(product_data)
            if extracted_info:
                # Add sitemap metadata
                extracted_info.update({
                    'sitemap_url': product_info['url'],
                    'lastmod': product_info['lastmod'],
                    'scraped_at': datetime.now().isoformat()
                })
                scraped_products.append(extracted_info)
            else:
                errors += 1
        else:
            errors += 1
        
        # Rate limiting
        if delay_between_requests > 0:
            time.sleep(delay_between_requests)
    
    print(f"✅ Scraping completed!")
    print(f"📊 Successfully scraped: {len(scraped_products)} products")
    print(f"❌ Errors encountered: {errors} products")
    
    # Step 6: Create DataFrame and save results
    if scraped_products:
        df = pd.DataFrame(scraped_products)
        
        # Save last update date
        current_time = datetime.now()
        save_last_update_date(current_time)
        
        print(f"💾 Results saved with {len(df)} products")
        print(f"📅 Next scrape will check for products updated after: {current_time}")
        
        return df
    else:
        print("❌ No products successfully scraped")
        return pd.DataFrame()

# 🧪 TEST THE UPDATED COMPREHENSIVE SCRAPER

print("🧪 Testing Updated Dekamarkt Comprehensive Scraper")
print("=" * 60)

# Test with a small sample first
print("📋 Starting test run with 3 products...")

# Run the comprehensive scraper
df_comprehensive_v2 = scrape_dekamarkt_comprehensive_v2(
    store_id=283,
    delay_between_requests=0.2,  # 200ms delay to be safe
    max_products=3,  # Test with just 3 products first
    force_full_scrape=True  # Ignore date filtering for test
)

if not df_comprehensive_v2.empty:
    print(f"\n📊 COMPREHENSIVE RESULTS OVERVIEW:")
    print(f"Total products scraped: {len(df_comprehensive_v2)}")
    print(f"Columns available: {len(df_comprehensive_v2.columns)}")
    
    # Show all available columns
    print(f"\n📋 ALL AVAILABLE COLUMNS:")
    for i, col in enumerate(df_comprehensive_v2.columns, 1):
        print(f"  {i:2d}. {col}")
    
    # Show sample product details
    if len(df_comprehensive_v2) > 0:
        print(f"\n🔍 SAMPLE PRODUCT DETAILS:")
        sample = df_comprehensive_v2.iloc[0]
        
        # Key product information
        print(f"Product ID: {sample.get('product_id', 'N/A')}")
        print(f"Name: {sample.get('header_text', 'N/A')}")
        print(f"Brand: {sample.get('brand', 'N/A')}")
        print(f"Price: €{sample.get('normal_price', 'N/A')}")
        print(f"Offer Price: €{sample.get('offer_price', 'N/A') if sample.get('offer_price') else 'No offer'}")
        print(f"Department: {sample.get('department', 'N/A')}")
        print(f"Description: {str(sample.get('description', 'N/A'))[:100]}...")
        
        # Nutritional info if available
        if sample.get('nutritional_values'):
            print(f"Nutritional Info: {str(sample.get('nutritional_values', ''))[:150]}...")
        
        # Ingredients if available
        if sample.get('ingredients'):
            print(f"Ingredients: {str(sample.get('ingredients', ''))[:100]}...")
            
else:
    print("❌ Test failed - no data retrieved")

🧪 Testing Updated Dekamarkt Comprehensive Scraper
📋 Starting test run with 3 products...
🏁 Starting Dekamarkt Comprehensive Scraper v2.0 (Fixed)
📥 Fetching product sitemap...
🔍 Parsing product URLs from sitemap...
📊 Found 11611 products in sitemap
🔄 Force full scrape enabled - processing all products
🎯 Limited to first 3 products for testing
🛒 Scraping detailed information for 3 products...
✅ Scraping completed!
📊 Successfully scraped: 3 products
❌ Errors encountered: 0 products
📅 Saved last update date: 2025-08-22 00:09:50.937537
💾 Results saved with 3 products
📅 Next scrape will check for products updated after: 2025-08-22 00:09:50.937537

📊 COMPREHENSIVE RESULTS OVERVIEW:
Total products scraped: 3
Columns available: 37

📋 ALL AVAILABLE COLUMNS:
   1. product_id
   2. header_text
   3. brand
   4. packaging
   5. is_weight_product
   6. max_amount
   7. department
   8. webgroup
   9. description
  10. additional_description
  11. normal_price
  12. offer_price
  13. is_single_use_pl

In [39]:
# 🎯 FINAL DEMONSTRATION - LARGER SAMPLE WITH CSV EXPORT

print("🎯 Final Demonstration - Larger Sample")
print("=" * 60)

# Run with more products to show scalability
print("📋 Running with 20 products and CSV export...")

# Run the comprehensive scraper with more products
df_final = scrape_dekamarkt_comprehensive_v2(
    store_id=283,
    delay_between_requests=0.15,  # 150ms delay for good balance
    max_products=20,  # Test with 20 products
    force_full_scrape=True  # Ignore date filtering for demo
)

if not df_final.empty:
    print(f"\n📊 FINAL RESULTS SUMMARY:")
    print(f"Total products scraped: {len(df_final)}")
    print(f"Data richness: {len(df_final.columns)} columns per product")
    
    # Save to CSV
    csv_filename = save_to_csv(df_final)
    
    # Show data quality metrics
    print(f"\n📈 DATA QUALITY METRICS:")
    
    key_fields = ['header_text', 'brand', 'normal_price', 'description', 
                 'nutritional_values', 'ingredients', 'main_image']
    
    for field in key_fields:
        if field in df_final.columns:
            non_null_count = df_final[field].notna().sum()
            percentage = (non_null_count / len(df_final)) * 100
            print(f"  {field}: {non_null_count}/{len(df_final)} ({percentage:.1f}% complete)")
    
    # Show product categories
    if 'department' in df_final.columns:
        dept_counts = df_final['department'].value_counts()
        print(f"\n🏪 PRODUCT CATEGORIES:")
        for dept, count in dept_counts.head(10).items():
            print(f"  {dept}: {count} products")
    
    # Show brand distribution
    if 'brand' in df_final.columns:
        brand_counts = df_final['brand'].value_counts()
        print(f"\n🏷️ TOP BRANDS:")
        for brand, count in brand_counts.head(5).items():
            if pd.notna(brand):
                print(f"  {brand}: {count} products")
    
    # Show price range
    if 'normal_price' in df_final.columns:
        prices = df_final['normal_price'].dropna()
        if not prices.empty:
            print(f"\n💰 PRICE RANGE:")
            print(f"  Min: €{prices.min():.2f}")
            print(f"  Max: €{prices.max():.2f}")
            print(f"  Average: €{prices.mean():.2f}")
    
    print(f"\n📄 SAMPLE PRODUCT NAMES:")
    for i, name in enumerate(df_final['header_text'].head(10), 1):
        if pd.notna(name):
            print(f"  {i:2d}. {name}")
            
    print(f"\n✅ SUCCESS! Comprehensive scraper ready for production use.")
    print(f"📁 Data saved to: {csv_filename}")
    print(f"🔄 Incremental updates: Enabled with lastmod date tracking")
    print(f"📊 Total discoverable products: 11,611")
    
else:
    print("❌ Final test failed")

print(f"\n{'='*60}")
print("🏁 Dekamarkt Comprehensive Scraper v2.0 - READY FOR PRODUCTION!")

🎯 Final Demonstration - Larger Sample
📋 Running with 20 products and CSV export...
🏁 Starting Dekamarkt Comprehensive Scraper v2.0 (Fixed)
📥 Fetching product sitemap...
🔍 Parsing product URLs from sitemap...
📊 Found 11611 products in sitemap
🔄 Force full scrape enabled - processing all products
🎯 Limited to first 20 products for testing
🛒 Scraping detailed information for 20 products...
✅ Scraping completed!
📊 Successfully scraped: 20 products
❌ Errors encountered: 0 products
📅 Saved last update date: 2025-08-22 00:11:03.777622
💾 Results saved with 20 products
📅 Next scrape will check for products updated after: 2025-08-22 00:11:03.777622

📊 FINAL RESULTS SUMMARY:
Total products scraped: 20
Data richness: 37 columns per product
💾 Saved 20 products to: dekamarkt_products_comprehensive_20250822_001103.csv

📈 DATA QUALITY METRICS:
  header_text: 20/20 (100.0% complete)
  brand: 20/20 (100.0% complete)
  normal_price: 18/20 (90.0% complete)
  description: 20/20 (100.0% complete)
  nutritio

In [ ]:
print("SITEMAP_URL:", SITEMAP_URL)
print("Testing sitemap access from notebook...")

import requests
try:
    resp = requests.get(SITEMAP_URL, timeout=30)
    print(f"Status: {resp.status_code}")
    if resp.status_code == 200:
        print(f"Content length: {len(resp.text)}")
        print("Sitemap accessible!")
    else:
        print(f"Error: {resp.status_code}")
except Exception as e:
    print(f"Exception: {e}")

SITEMAP_URL: https://www.dekamarkt.nl/products-sitemap.xml
Testing sitemap access from notebook...
Status: 403
Error: 403


: 